# Data Collection


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import altair as alt
from sklearn.model_selection import train_test_split

df_dtypes = {
    'category': 'str',
    'title': 'str',
    'body': 'str',
    'amenities': 'str',
    'currency': 'str',
    'fee': 'str',
    'has_photo': 'str',
    'pets_allowed': 'str',
    'price_display': 'str',
    'price_type': 'str',
    'address': 'str',
    'cityname': 'str',
    'state': 'str',
    'source': 'str'
}
df = pd.read_csv("./properties_us.csv", sep=';', encoding='ISO-8859-1', dtype=df_dtypes)
train_set, test_set = train_test_split(df, test_size=.2, random_state=42)
print(df.dtypes)

id                 int64
category          object
title             object
body              object
amenities         object
bathrooms        float64
bedrooms         float64
currency          object
fee               object
has_photo         object
pets_allowed      object
price            float64
price_display     object
price_type        object
square_feet        int64
address           object
cityname          object
state             object
latitude         float64
longitude        float64
source            object
time               int64
dtype: object


# Data Cleaning

Lot of redundant features. 'body' contains text-description of property, 'currency' is predominately USD, 'fee' is >99% no and i don't even know what it means, 'source' is whatever web-platform the data was scraped from. Additionally i'll want to convert whatever string features into numerical ones.


In [3]:
df_clean = df.drop(['body', 'currency', 'fee', 'price_display', 'source', 'address'], axis=1)

def PetCount (x):
    i = 0
    if isinstance(x, str):
        i+=1+ x.count(',')
    return i

# Has photo Yes=1, No=0
df_clean['has_photo'] = (df_clean['has_photo'] != 'No').astype(int)
# Count num of pettypes allowed
df_clean['pets_allowed'] = df_clean['pets_allowed'].replace({pd.NaT: 0, None: 0})
df_clean['pets_allowed'] = df_clean['pets_allowed'].apply(lambda x: PetCount(x))
# Don't like imperial units
df_clean['square_meters'] = (df_clean['square_feet'] * 0.09290304).round(0)
df_clean = df_clean.drop('square_feet', axis=1)
# Drop the 4 records that are not monthly payment
df_clean = df_clean[df_clean['price_type'] == 'Monthly']
df_clean = df_clean.drop('price_type', axis=1)
df_clean['amenities'] = df_clean['amenities'].replace({np.nan: None})

df_clean.isna().sum()


id                   0
category             0
title                0
amenities        16044
bathrooms           62
bedrooms           123
has_photo            0
pets_allowed         0
price                1
cityname           302
state              302
latitude            25
longitude           25
time                 0
square_meters        0
dtype: int64

We're gonna drop some of the records with missing entries: bathrooms, bedrooms, cityname, state, latitude, longitude


In [4]:
df_filtered = df_clean.dropna(subset=['bathrooms', 'bedrooms', 'cityname', 'state', 'latitude', 'longitude', 'price'])
df_filtered.isna().sum()

id                   0
category             0
title                0
amenities        15841
bathrooms            0
bedrooms             0
has_photo            0
pets_allowed         0
price                0
cityname             0
state                0
latitude             0
longitude            0
time                 0
square_meters        0
dtype: int64

In [5]:
df_filtered.describe()

,id,bathrooms,bedrooms,has_photo,pets_allowed,price,latitude,longitude,time,square_meters
count,9.900400e+04,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,9.900400e+04,99004.000000
mean,5.357983e+09,1.445861,1.728678,0.907276,0.765868,1525.582229,36.941553,-91.544693,1.559645e+09,88.812987
std,1.846400e+08,0.547094,0.748669,0.290046,0.961985,898.693022,4.603604,15.825449,1.104714e+07,33.936311
min,5.121046e+09,1.000000,0.000000,0.000000,0.000000,100.000000,19.573800,-159.369800,1.544174e+09,10.000000
25%,5.197947e+09,1.000000,1.000000,1.000000,0.000000,1013.750000,33.743500,-104.817100,1.550832e+09,68.000000
50%,5.508672e+09,1.000000,2.000000,1.000000,0.000000,1350.000000,37.213900,-84.546500,1.568745e+09,84.000000
75%,5.509006e+09,2.000000,2.000000,1.000000,2.000000,1795.000000,39.955900,-77.576700,1.568767e+09,104.000000
max,5.669439e+09,9.000000,9.000000,1.000000,3.000000,52500.000000,64.833200,-68.778800,1.577391e+09,1115.000000


In [6]:
import altair as alt
alt.data_transformers.enable("vegafusion")

df_numeric = df_filtered.select_dtypes(include='number')
plots = []
for column in df_numeric.columns:
    plot = alt.Chart(df_numeric).mark_bar().encode(
        alt.X(f'{column}' + ":Q", bin=True),
        y='count()'
        )
    plots.append(plot)        

    



# plot_bathrooms = alt.Chart(df_filtered).mark_bar().encode(
#     alt.X("bathrooms:Q", bin=False),
#     y='count()',
# )
# plot_bedrooms = alt.Chart(df_filtered).mark_bar().encode(
#     alt.X("bedrooms:Q", bin=False),
#     y='count()',
# )
# # Not histogram
# plot_has_photo = alt.Chart(df_filtered).mark_bar().encode(
#     x='has_photo',
#     y='count()',
# )
# plot_pets_allowed = alt.Chart(df_filtered).mark_bar().encode(
#     x='pets_allowed',
#     y='count()',
# )
# plot_price = alt.Chart(df_filtered).mark_bar().encode(
#     alt.X("price:Q", bin=False),
#     y='count()',
# )
# plot_latitude = alt.Chart(df_filtered).mark_bar().encode(
#     alt.X("latitude:Q", bin=False),
#     y='count()',
# )
# plot_longitude = alt.Chart(df_filtered).mark_bar().encode(
#     alt.X("longitude:Q", bin=False),
#     y='count()',
# )
# plot_time = alt.Chart(df_filtered).mark_bar().encode(
#     alt.X("time:Q", bin=False),
#     y='count()',
# )
# plot_square_meters = alt.Chart(df_filtered).mark_bar().encode(
#     alt.X("square_meters:Q", bin=False),
#     y='count()',
# )

grid = alt.vconcat(

    alt.hconcat(plots[1], plots[2], plots[3]),
    alt.hconcat(plots[4], plots[5], plots[6]),
    alt.hconcat(plots[7], plots[8], plots[9]),

)

grid.display()

alt.VConcatChart(...)

In [ ]:
import plotly.figure_factory as ff

# VScode keeps crashing with the full dataset so we'll take a sample just for visualization

df_numeric = df_sampled.select_dtypes(include='number')
Q1 = df_numeric['price'].quantile(.25)
Q3 = df_numeric['price'].quantile(.75)
IQR = Q3 - Q1
upper_fence = Q3 + 3*IQR
df_omitted_outliers = df_numeric[df_numeric['price'] <= upper_fence]   

df_aggregate =


label = ['Distribution'] # name of the dataset

# plots = []
# for column in df_omitted_outliers:
#     fig = ff.create_distplot([df_omitted_outliers[column]], label, bin_size=30)
#     plots.append(fig)
# plots[0].show()